In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
import tensorflow as tf

2024-08-13 16:04:27.755855: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 16:04:27.771866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-13 16:04:27.776704: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-13 16:04:27.788286: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-13 16:04:28.711540: W tensorflow/comp

In [2]:
batch_size = 64
epochs = 200
latent_dim = 256
num_samples = 10000

In [3]:
data_path = 'dataset/fra.txt'

In [4]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path,'r',encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples,len(lines)-1)]:
    input_text, target_text,_= line.split('\t')
    target_text = '\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [21]:
input_texts

['Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I know.',
 'I left.',
 'I left.',
 'I lied.',
 'I lost.',
 'I paid.',
 "I'm 19.",
 "I'm OK.",
 "I'm OK.",
 'Listen.',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'No way!',
 'Really?',
 'Really?',
 'Really?',
 'Thanks.',
 'We try.',
 'We won.',
 'We won.',
 'We won.',
 'We won.',
 'Ask Tom.',
 'Awesome!',
 'Be calm.',
 'Be calm.',
 'Be calm.',
 'Be cool.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fair.',
 'Be fai

In [22]:
with open('output.txt', 'w') as file:
    for item in input_texts:
        file.write(f"{item}\n")

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_len = max([len(txt) for txt in input_texts])
max_decoder_seq_len = max([len(txt) for txt in target_texts])

In [6]:
print('num of samples:', len(input_texts))
print('num of unique input tokens:', num_encoder_tokens)
print('num of unique output tokens:', num_decoder_tokens)
print('max seq len for inputs:', max_encoder_seq_len)
print('max seq len for outputs:', max_decoder_seq_len)

num of samples: 10000
num of unique input tokens: 71
num of unique output tokens: 92
max seq len for inputs: 16
max seq len for outputs: 59


In [7]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)]
)

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts),max_encoder_seq_len, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),    
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_len, num_decoder_tokens),
    dtype = 'float32')

In [9]:
encoder_input_data.shape

(10000, 16, 71)

In [10]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1
    encoder_input_data[i, t+1:, input_token_index[' ']] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]]=1
        if t>0:
            decoder_target_data[i, t-1, target_token_index[char]]=1
    decoder_input_data[i,t+1:, target_token_index[' ']] = 1
    decoder_target_data[i, t:, target_token_index[' ']]=1

In [11]:
encoder_input_data[0].shape

(16, 71)

# Model

In [12]:
with tf.device('/cpu:0'):
    encoder_inputs = Input(shape=(None, num_encoder_tokens))
    encoder = LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

2024-08-13 16:04:44.862619: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9470 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:65:00.0, compute capability: 7.5


In [13]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,_,_ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [14]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [15]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, 
         batch_size = batch_size,
         epochs=epochs,
         validation_split=0.2)

Epoch 1/200


2024-08-13 16:04:51.363890: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6931 - loss: 1.6027 - val_accuracy: 0.6972 - val_loss: 1.1523
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7340 - loss: 1.0020 - val_accuracy: 0.7013 - val_loss: 1.1310
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7511 - loss: 0.9047 - val_accuracy: 0.7242 - val_loss: 0.9320
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.7741 - loss: 0.8068 - val_accuracy: 0.7624 - val_loss: 0.8228
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7950 - loss: 0.7144 - val_accuracy: 0.7763 - val_loss: 0.7713
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8054 - loss: 0.6700 - val_accuracy: 0.7857 - val_loss: 0.7305
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8134 - loss: 0.6360 - val_accuracy: 0.7923 - val_loss: 0.7029
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8214 - loss: 0.6081 - val_accuracy

In [17]:
encoder_model = Model(encoder_inputs,encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state= decoder_states_inputs)
decoder_states=[state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)


In [18]:
reverse_input_char_index= dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index= dict((i, char) for char, i in target_token_index.items())

In [20]:
def decode_sequence(input_seq):
    
    states_value = encoder_model.predict(input_seq, verbose=0)
    
    target_seq = np.zeros((1,1, num_decoder_tokens))
    
    target_seq[0, 0, target_token_index['\t']]=1.
    
    stop_condition= False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h,c = decoder_model.predict([target_seq]+states_value, verbose=0)
        sampled_token_index = np.argmax(output_tokens[0,-1,:])
       
        sampled_char = reverse_target_char_index[sampled_token_index]
        
        decoded_sentence += sampled_char
        if (sampled_char=='\n' or len(decoded_sentence)>max_decoder_seq_len):
            stop_condition=True

        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0,0, sampled_token_index]=1.
        
        states_value = [h, c]
    return decoded_sentence

for seq_index in range(100):
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decode_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence  :', input_texts[seq_index])
    print('Decoded Sentence:', decode_sentence)

-
Input sentence  : Go.
Decoded Sentence: Saute.

-
Input sentence  : Hi.
Decoded Sentence: Salut !

-
Input sentence  : Hi.
Decoded Sentence: Salut !

-
Input sentence  : Run!
Decoded Sentence: Cours !

-
Input sentence  : Run!
Decoded Sentence: Cours !

-
Input sentence  : Who?
Decoded Sentence: Qui ?

-
Input sentence  : Wow!
Decoded Sentence: Lave-toi !

-
Input sentence  : Fire!
Decoded Sentence: Au feu !

-
Input sentence  : Help!
Decoded Sentence: Aide-moi.

-
Input sentence  : Jump.
Decoded Sentence: Maute-moi.

-
Input sentence  : Stop!
Decoded Sentence: Stop !

-
Input sentence  : Stop!
Decoded Sentence: Stop !

-
Input sentence  : Stop!
Decoded Sentence: Stop !

-
Input sentence  : Wait!
Decoded Sentence: Attends !

-
Input sentence  : Wait!
Decoded Sentence: Attends !

-
Input sentence  : Go on.
Decoded Sentence: Allez-y maintenant.

-
Input sentence  : Go on.
Decoded Sentence: Allez-y maintenant.

-
Input sentence  : Go on.
Decoded Sentence: Allez-y maintenant.

-
Input se